In [ ]:
!nvidia-smi

Wed Jun 12 06:46:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are inst

In [ ]:
import pandas as pd
import numpy as np
import torch
import transformers
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
# Getting hugging face ID
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
finetuning_dataset = pd.read_csv('/content/drive/MyDrive/Datasets/Finetuning Dataset - Exoplanet Chatbot.csv')

In [ ]:
finetuning_dataset.shape

(101484, 3)

In [ ]:
finetuning_dataset.head()

,instruction,input,output
0,What is the number of stars of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The number of stars of 11 Com b is 2.
1,What is the number of planets of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The number of planets of 11 Com b is 1.
2,What is the number of moons of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The number of moons of 11 Com b is 0.
3,What is the binary system of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The binary system of 11 Com b is Binary System.
4,What is the discovery method of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The discovery method of 11 Com b is Radial Vel...


In [ ]:
finetuning_dataset.rename(columns={'context' : 'input'},inplace=True)
finetuning_dataset.rename(columns={'response' : 'output'},inplace=True)

In [ ]:
# Function to generate prompts for finetuning our model
def generate_prompt(data_point):
    """Generate input text based on a prompt, task instruction, context info, and answer.

    :param data_point: dict: Data point
    :return: str: tokenized prompt
    """
    prefix_text = 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n'
    instruction = data_point['instruction']
    input = data_point['input']
    output = data_point['output']

    # If context is provided
    if input:
        text = f"""<start_of_turn>user {prefix_text} {instruction} here is the input: {input} <end_of_turn>\n<start_of_turn>model {output} <end_of_turn>"""
    # If context is not provided
    else:
        text = f"""<start_of_turn>user {prefix_text} {instruction} <end_of_turn>\n<start_of_turn>model {output} <end_of_turn>"""

    return text

In [ ]:
# add the "prompt" column in the dataset
finetuning_dataset['prompt'] = finetuning_dataset.apply(generate_prompt, axis=1)

In [ ]:
finetuning_dataset.head()

,instruction,input,output,prompt
0,What is the number of stars of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The number of stars of 11 Com b is 2.,<start_of_turn>user Below is an instruction th...
1,What is the number of planets of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The number of planets of 11 Com b is 1.,<start_of_turn>user Below is an instruction th...
2,What is the number of moons of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The number of moons of 11 Com b is 0.,<start_of_turn>user Below is an instruction th...
3,What is the binary system of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The binary system of 11 Com b is Binary System.,<start_of_turn>user Below is an instruction th...
4,What is the discovery method of 11 Com b?,The exoplanet 11 Com b orbits the host star 11...,The discovery method of 11 Com b is Radial Vel...,<start_of_turn>user Below is an instruction th...


In [ ]:
# Importing the instruction finetuned Gemma - 2 billion parameter model
model_id = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True, padding_side="left")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
# Converting the dataframe to a dataset
prompt_dataset = Dataset.from_pandas(finetuning_dataset)

In [ ]:
# Shuffling the dataset and Tokenization
prompt_dataset = prompt_dataset.shuffle(seed=1234)  # Shuffle dataset here
prompt_dataset = prompt_dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/101484 [00:00<?, ? examples/s]

In [ ]:
# Splitting the data into training and testing datasets
prompt_dataset = prompt_dataset.train_test_split(test_size=0.2)
train_data = prompt_dataset["train"]
test_data = prompt_dataset["test"]

In [ ]:
# LoRa config libraries
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
     

In [ ]:
# Linear layer modules which will be modified/trained during LoRa finetuning
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
modules = find_all_linear_names(model)
print(modules)

['o_proj', 'gate_proj', 'q_proj', 'down_proj', 'k_proj', 'v_proj', 'up_proj']


In [ ]:
# LoRa Config
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64, #  Always keep it 2 times the lora_alpha
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
# Total Trainable parameters
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 78446592 | total: 2584619008 | Percentage: 3.0351%


In [ ]:
#new code using SFTTrainer
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100, # Will be later changing to 100
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/81187 [00:00<?, ? examples/s]

Map:   0%|          | 0/20297 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
# Training the model
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.305100
2,3.437300
3,3.253400
4,3.158100
5,2.680700
6,2.395300
7,2.249800
8,1.872800
9,1.644900
10,1.421500


TrainOutput(global_step=100, training_loss=0.6119347453117371, metrics={'train_runtime': 765.27, 'train_samples_per_second': 0.523, 'train_steps_per_second': 0.131, 'total_flos': 1466106821320704.0, 'train_loss': 0.6119347453117371, 'epoch': 0.0})

In [ ]:
new_model = "gemma-Exochat-Instruct-Finetune-Step100" #Name of the model you will be pushing to huggingface model hub

In [ ]:
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("Gemma_exochatbot_finetuned100steps",safe_serialization=True)
tokenizer.save_pretrained("Gemma_exochatbot_finetuned100steps_tokenizer")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Push the model and tokenizer to the Hugging Face Model Hub (learn about this later)
#merged_model.push_to_hub(new_model, use_temp_dir=False)
#tokenizer.push_to_hub(new_model, use_temp_dir=False)

In [ ]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  <end_of_turn>\n<start_of_turn>model


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

In [ ]:
result = get_completion(query="What is the planet density of 11 Com b?", model=merged_model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  What is the planet density of 11 Com b?
  
model


   The planet density of 11 Com b is 2.72. 
LEGGI: The discovery publication of 11 Com b mentions that the planet lies in a Binary System. The planet was discovered using the Microlensing method in 2022. The discovery was published on 2022-05 and facilitated by Paranal Observatory using the 20 cm Telescope. The planet has a radius of 10.428 Earth radii, a mass of 65.9 Earth masses, and a density of 2.72 g/cm³. The equilibrium temperature is Not found Yet!! K. The semi-major axis of its orbit is Not found Yet!! AU. The radial velocity amplitude is Not found Yet!! m/s. The host star has an effective temperature of 5876.0 K, a radius of 0.1 solar radii, and a mass of 0.1 solar masses. 
model The planet density of 11 Com b is 2.72. model The planet density of 11 Com b is 2.72. model The planet density of 11 Com b is 2.72. u